# Testing thershold

In [ ]:
from glob import glob

import numpy as np

from polyphys.manage.parser import HnsCyl
from polyphys.analyze.clusters import generate_contact_matrix, find_direct_contacts, hns_binding

In [ ]:
probe_path = "/Users/amirhsi_mini/research_data/hns_cyl-trjs/N*/N*distMatMonPatchT.npy"
paths = sorted(glob(probe_path))
print(paths)
lj_cut = np.round(2**(1/6),6)
dmon = 1
dhpatch = 0.178
rcut_m_hpatch_min = np.round(0.5 * (dmon + dhpatch), 3) 
print(rcut_m_hpatch_min)
rcut_m_hpatch_max = np.round(0.5 * lj_cut * (dmon + dhpatch), 3)
print(rcut_m_hpatch_max)
print(rcut_m_hpatch_max-rcut_m_hpatch_min)
print(rcut_m_hpatch_min)
rcut_m_hpatch_range = np.arange(rcut_m_hpatch_min, rcut_m_hpatch_max, 0.009)
print(rcut_m_hpatch_range)

In [ ]:
bindings = {
    'dcrowd': [],
    'phi_c_bulk': [],
    'nhns': [],
    'rcut': [],
    'n_m_hpatch_bound': [],
    'n_hpatch_free': [],
    'n_hpatch_engaged': [],
    'n_hcore_free': [],
    'n_hcore_bridge': [],
    'n_hcore_dangle': [],
    'n_mon_bound': [],
    'n_mon_cis': [],
    'n_mon_trans': []
}
m_m_gen_dist_dict = {
    'dcrowd': [],
    'phi_c_bulk': [],
    'nhns': [],
    'rcut': [],
    'loop_size_hist': []   
}
for p in paths:
    dist_m_hpatch_t = np.load(p)
    print(p)
    sim_info = HnsCyl(
        p,
        'whole',
        'cylindrical',
        'nucleoid',
        'ring'
    )
    for rcut in rcut_m_hpatch_range[-1:]:
        print(rcut)
        results = {
            'n_m_hpatch_bound': [],
            'n_hpatch_free': [],
            'n_hpatch_engaged': [],
            'n_hcore_free': [],
            'n_hcore_bridge': [],
            'n_hcore_dangle': [],
            'n_mon_bound': [],
            'n_mon_cis': [],
            'n_mon_trans': []
        }
        if sim_info.topology == 'linear':
            loop_size_hist = np.zeros(sim_info.nmon, dtype=int)
        elif sim_info.topology == 'ring':
            loop_size_hist = np.zeros((sim_info.nmon//2)+1, dtype=int)
        else:
            raise ValueError(
                f"The genomic distance is not defined for '{topology}' topology"
                )
        bindings['dcrowd'].append(sim_info.dcrowd)
        bindings['phi_c_bulk'].append(round(sim_info.phi_c_bulk,3))
        bindings['nhns'].append(sim_info.nhns)
        bindings['rcut'].append(rcut)
        m_m_gen_dist_dict['dcrowd'].append(sim_info.dcrowd)
        m_m_gen_dist_dict['phi_c_bulk'].append(round(sim_info.phi_c_bulk,3))
        m_m_gen_dist_dict['nhns'].append(sim_info.nhns)
        m_m_gen_dist_dict['rcut'].append(rcut)
        n_frames = len(dist_m_hpatch_t)
        for mat in dist_m_hpatch_t:
            direct_contact_m_hpatch = find_direct_contacts(mat, rcut, inclusive=False)
            results, loop_size_hist = hns_binding(
                direct_contact_m_hpatch, sim_info.topology, results=results, loop_length_hist=loop_size_hist)
        bindings['n_m_hpatch_bound'].append(np.sum(
            np.array(results['n_m_hpatch_bound'], dtype=int))/n_frames
                                         )
        bindings['n_hpatch_engaged'].append(np.sum(
            np.array(results['n_hpatch_engaged'], dtype=int))/n_frames
                                           )
        bindings['n_hpatch_free'].append(np.sum(
            np.array(results['n_hpatch_free'], dtype=int))/n_frames
                                        )
        bindings['n_hcore_free'].append(np.sum(
            np.array(results['n_hcore_free'], dtype=int))/n_frames
                                       )
        bindings['n_hcore_bridge'].append(np.sum(
            np.array(results['n_hcore_bridge'], dtype=int))/n_frames
                                         )
        bindings['n_hcore_dangle'].append(np.sum(
            np.array(results['n_hcore_dangle'], dtype=int))/n_frames
                                         )
        bindings['n_mon_bound'].append(np.sum(np.array(results['n_mon_bound'], dtype=int))/n_frames)
        bindings['n_mon_cis'].append(np.sum(np.array(results['n_mon_cis'], dtype=int))/n_frames)
        bindings['n_mon_trans'].append(np.sum(np.array(results['n_mon_trans'], dtype=int))/n_frames)
        m_m_gen_dist_dict['loop_size_hist'].append(loop_size_hist/n_frames)

In [ ]:
with open('HnsCyl-choice_of_thershold.pkl', 'wb') as f:
    pickle.dump(bindings, f)

In [ ]:
# find a particular type of binding:
indices = np.where(contact_per_hpatch_t==3)
indices = np.array(indices).T
select_inds = indices[-2]
print(select_inds)
print(contact_per_hpatch_t[select_inds[0]])
print(np.take(contact_per_hpatch_t,select_inds))

# viz

In [ ]:
from glob import glob
import pickle 

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
lj_cut = np.round(2**(1/6),6)
dmon = 1
dhpatch = 0.178
rcut_m_hpatch_min = np.round(0.5 * (dmon + dhpatch), 3) 
print(rcut_m_hpatch_min)
rcut_m_hpatch_max = np.round(0.5 * lj_cut * (dmon + dhpatch), 3)
print(rcut_m_hpatch_max)
print(rcut_m_hpatch_max-rcut_m_hpatch_min)
print(rcut_m_hpatch_min)
rcut_m_hpatch_range = np.arange(rcut_m_hpatch_min, rcut_m_hpatch_max, 0.009)
rcut_m_hpatch_range

In [ ]:
with open('../../Datasets/HnsCyl-choice_of_thershold.pkl', 'rb') as f:
    loaded_dict = pickle.load(f)

## df from loop lengths histogram:
bindings_smaller = loaded_dict.copy()
loop_size_hist = {}
nframes = 20001
nmon = 101
for i in range(nmon):
    loop_size_hist[i] = []
for key, value in bindings_smaller.items():
    if key == 'results':
        pass
    else:
        loop_size_hist[key] = value
nens = len(loop_size_hist['dcrowd'])
for i in range(nens):
    hist_dummy = bindings_smaller['results'][i]['loop_size_hist']
    for j in range(nmon):
        loop_size_hist[j].append(hist_dummy[j])

loop_size_hist_df = pd.DataFrame.from_dict(loop_size_hist)
id_vars = ['dcrowd', 'phi_c_bulk', 'nhns', 'rcut']
value_vars = loop_size_hist_df.columns[:nmon]
loop_size_hist_melt = pd.melt(loop_size_hist_df, id_vars=id_vars, value_vars=value_vars, var_name='genomic_distance', value_name='frequency')
loop_size_hist_melt['frequency'] = loop_size_hist_melt['frequency'] / nframes
loop_size_hist_melt['frequency'] = loop_size_hist_melt.groupby(['dcrowd', 'phi_c_bulk', 'nhns', 'rcut'])['frequency'].transform(lambda x: x / x.max())

In [ ]:
# Plotting loop length statistics
height = 3
aspect = 2 * 1.618 # Golden ratio
font_family = 'Times New Roman'#'DejaVu Sans'#'sans-serif' # 'Times New Roman'
#math_font = 'dejavuserif' #'stixsans'
font_scale = 1
plot_context = 'talk'
save_to = './'
ext = 'pdf'
color_palette = 'tab10'
rc_params = {
    #'mathtext.fontset': math_font, #'regular',
    #'mathtext.default': 'regular',
    'axes.formatter.use_mathtext': True,
    "font.family": font_family,  # use serif/main font for text elements
}
facet_kws = {
    'sharey': False,
    'sharex': True,
    'legend_out': True
}
legend_kws = {
    'ncol': 1,
    'frameon': True,
    'facecolor': 'white',
    'framealpha': 1.0,
    'edgecolor': 'black',
    'columnspacing': 1,
    'borderpad': 0.2,
    'handletextpad': 0.1,
    'markerscale': 3
}
fig_title_kws = {'x': 0.5, 'y': 0.97, 'ha': 'center'}
sns.set_theme(
    context=plot_context,
    style='ticks',
    palette=color_palette,
    font=font_family,
    font_scale=font_scale,
    rc=rc_params
)
col_order = sorted(loop_size_hist_melt['nhns'].unique())
loop_size_hist_melt_trunc = loop_size_hist_melt.loc[(loop_size_hist_melt['genomic_distance']>2)&(loop_size_hist_melt['genomic_distance']<60),:]
f_grid = sns.relplot(
    data=loop_size_hist_melt_trunc,
    x='genomic_distance',
    y='frequency',
    col='nhns',
    hue='rcut',
    col_order=sorted(loop_size_hist_melt['nhns'].unique()),
    col_wrap=1,
    height=height,
    aspect=aspect,
    palette=color_palette,
    legend='full',
    kind='line',
    alpha=0.8,
    facet_kws=facet_kws
)

#f_grid.set_titles(r'$n_{{hns}}={col_name}$')
f_grid.set_titles('')

for idx, (ax, col_name) in enumerate(zip(f_grid.axes.flat,col_order)):
    ax.text(0.1,0.8, fr"$n_{{hns}}={col_name}$", transform=ax.transAxes)

f_grid.set_axis_labels(r'$|\Delta n_{ij}|$', r'$\mathcal{P}(|\Delta n_{ij}|>2)$')

#f_grid.set(xticks=rcut_m_hpatch_range)
f_grid.fig.suptitle(r'$N=200, D/a_m=8, a_c/a_m=2, \phi_c=0.097$', **fig_title_kws)
f_grid.tight_layout(w_pad=0.2, h_pad=0.2)
f_grid.legend.set_title(r'$r_{thers}$')
#f_grid.set(yscale='log')
f_grid.savefig('loop_length_distribution.pdf', bbox_inches='tight', dpi=300)

In [ ]:
with open('../../Datasets/HnsCyl-choice_of_thershold.pkl', 'rb') as f:
    loaded_dict = pickle.load(f)


## df from loop lengths histogram:
bindings_smaller = loaded_dict.copy()
del bindings_smaller['results']
# binding dataframe
binding_df = pd.DataFrame.from_dict(bindings_smaller)
nframes = 20001
nmon= 200
for col in ['n_m_hpatch_bound',
       'n_hpatch_free', 'n_hpatch_engaged', 'n_hcore_free', 'n_hcore_bridge',
       'n_hcore_dangle', 'n_mon_bound', 'n_mon_cis', 'n_mon_trans']:
    binding_df[col] = binding_df[col]
for col in ['n_hcore_free', 'n_hcore_bridge', 'n_hcore_dangle']:
    binding_df[col] = binding_df[col] / binding_df['nhns']
for col in ['n_hpatch_free', 'n_hpatch_engaged']:
    binding_df[col] = 0.5 * binding_df[col] / binding_df['nhns']
for col in ['n_mon_bound', 'n_mon_cis', 'n_mon_trans']:
    binding_df[col] =binding_df[col] / nmon
binding_df['n_hpatch_free'] = 2 - binding_df['n_hpatch_engaged']
id_vars = ['dcrowd', 'phi_c_bulk', 'nhns', 'rcut']
#value_vars = ['n_hcore_free', 'n_hcore_bridge', 'n_hcore_dangle']
#value_vars = ['n_mon_bound', 'n_mon_cis', 'n_mon_trans']
#value_vars = ['n_hpatch_free', 'n_hpatch_engaged']
#binding_melt = pd.melt(binding_df, id_vars=id_vars, value_vars=value_vars, var_name='binding_types', value_name='averaged_bound_hns')

In [ ]:
# plot: h patches
value_vars = ['n_hpatch_free', 'n_hpatch_engaged']
binding_melt = pd.melt(binding_df, id_vars=id_vars, value_vars=value_vars, var_name='binding_types', value_name='averaged_bound_hns')
height = 7
aspect = 1.618 # Golden ratio
font_family = 'Times New Roman'#'DejaVu Sans'#'sans-serif' # 'Times New Roman'
#math_font = 'dejavusans' #'dejavuserif' #'stixsans'
font_scale = 1.4
plot_context = 'talk'
save_to = './'
ext = 'pdf'
color_palette = 'tab10'
rc_params = {
    #'mathtext.fontset': math_font, #'regular',
    #'mathtext.default': 'regular',
    'axes.formatter.use_mathtext': True,
    "font.family": font_family,  # use serif/main font for text elements
}
legend_kws = {
    'ncol': 1,
    'frameon': True,
    'facecolor': 'white',
    'framealpha': 1.0,
    'edgecolor': 'black',
    'columnspacing': 1,
    'borderpad': 0.2,
    'handletextpad': 0.1,
    'markerscale': 3
}
fig, ax = plt.subplots(1,1, figsize=(aspect*height,height)) 
sns.set_theme(
    context=plot_context,
    style='ticks',
    palette=color_palette,
    font=font_family,
    font_scale=font_scale,
    rc=rc_params
)
hue_order = list(binding_melt['binding_types'].unique())
hue_order = ['n_hpatch_free', 'n_hpatch_engaged']
col_order = sorted(binding_melt['nhns'].unique())
f_grid = sns.lineplot(
    data=binding_melt,
    x='rcut',
    y='averaged_bound_hns',
    hue='binding_types',
    hue_order=hue_order,
    style='nhns',
    style_order=col_order,
    legend='full',
    dashes=True,
    markers=True,
    estimator=None,
    markersize=15,
    alpha=0.8,
    ax=ax
)
ax.axvline(x=rcut_m_hpatch_range[0], color='black',label=r'$r_{thers}=(a_m+a_{h,patch})/2$', ls='--', zorder=0)
ax.axvline(x=rcut_m_hpatch_range[-1], color='black',label=r'$r_{thers}=2^{1/6}(a_m+a_{h,patch})/2$',ls='-.', zorder=0)
f_grid.set_title(r'$N=200, D/a_m=8, a_c/a_m=2, \phi_c=0.097$', y=1.01)
f_grid.set_xlabel(r'$r_{thers}$')
f_grid.set_ylabel(r'$\langle n_{hns, patch}^{bound}\rangle /n_{hns, patch}$')
f_grid.set(xticks=rcut_m_hpatch_range)
handles, labels = ax.get_legend_handles_labels()
legend_labels = ['Binding types'] + ['Free H-NS patch', 'Bound H-NS patch'] + [r'$n_{hns}$'] + col_order + labels[-2:]
ax.legend(handles, legend_labels, loc='upper left', bbox_to_anchor=(1,1), frameon=False)
#ax.legend()
#f_grid.set(yscale='log')
#fig.tight_layout()#w_pad=0.2, h_pad=0.2)
fig.savefig('choice_of_thershold-hpatches.pdf', dpi=300,  bbox_inches='tight')

In [ ]:
# Figure: h cores
value_vars = ['n_hcore_free', 'n_hcore_bridge', 'n_hcore_dangle']
binding_melt = pd.melt(binding_df, id_vars=id_vars, value_vars=value_vars, var_name='binding_types', value_name='averaged_bound_hns')
height = 7
aspect = 1.618 # Golden ratio
font_family = 'Times New Roman'#'DejaVu Sans'#'sans-serif' # 'Times New Roman'
#math_font = 'dejavusans' #'dejavuserif' #'stixsans'
font_scale = 1.4
plot_context = 'talk'
save_to = './'
ext = 'pdf'
color_palette = 'tab10'
rc_params = {
    #'mathtext.fontset': math_font, #'regular',
    #'mathtext.default': 'regular',
    'axes.formatter.use_mathtext': True,
    "font.family": font_family,  # use serif/main font for text elements
}
legend_kws = {
    'ncol': 1,
    'frameon': True,
    'facecolor': 'white',
    'framealpha': 1.0,
    'edgecolor': 'black',
    'columnspacing': 1,
    'borderpad': 0.2,
    'handletextpad': 0.1,
    'markerscale': 3
}
fig, ax = plt.subplots(1,1, figsize=(aspect*height,height)) 
sns.set_theme(
    context=plot_context,
    style='ticks',
    palette=color_palette,
    font=font_family,
    font_scale=font_scale,
    rc=rc_params
)
hue_order = list(binding_melt['binding_types'].unique())
hue_order = ['n_hcore_free', 'n_hcore_dangle', 'n_hcore_bridge']
col_order = sorted(binding_melt['nhns'].unique())
f_grid = sns.lineplot(
    data=binding_melt,
    x='rcut',
    y='averaged_bound_hns',
    hue='binding_types',
    hue_order=hue_order,
    style='nhns',
    style_order=col_order,
    legend='full',
    dashes=True,
    markers=True,
    estimator=None,
    markersize=15,
    alpha=0.8,
    ax=ax
)
ax.axvline(x=rcut_m_hpatch_range[0], color='black',label=r'$r_{thers}=(a_m+a_{h,patch})/2$', ls='--', zorder=0)
ax.axvline(x=rcut_m_hpatch_range[-1], color='black',label=r'$r_{thers}=2^{1/6}(a_m+a_{h,patch})/2$',ls='-.', zorder=0)
f_grid.set_title(r'$N=200, D/a_m=8, a_c/a_m=2, \phi_c=0.097$', y=1.01)
f_grid.set_xlabel(r'$r_{thers}$')
f_grid.set_ylabel(r'$\langle n_{hns}^{bound}\rangle /n_{hns}$')
f_grid.set(xticks=rcut_m_hpatch_range)
handles, labels = ax.get_legend_handles_labels()
legend_labels = ['Binding types'] + ['Free H-NS', 'Dangling', 'Bridging'] + [r'$n_{hns}$'] + col_order + labels[-2:]
ax.legend(handles, legend_labels, loc='upper left', bbox_to_anchor=(1,1), frameon=False)
#ax.legend()
#f_grid.set(yscale='log')
fig.savefig('choice_of_thershold-hns_cores.pdf', dpi=300,  bbox_inches='tight')

In [ ]:
#Figure: cis and trans monomers
value_vars = ['n_mon_bound', 'n_mon_cis', 'n_mon_trans']
binding_melt = pd.melt(binding_df, id_vars=id_vars, value_vars=value_vars, var_name='binding_types', value_name='averaged_bound_hns')
height = 5
aspect = 0.75*1.618 # Golden ratio
font_family = 'Times New Roman'#'DejaVu Sans'#'sans-serif' # 'Times New Roman'
#math_font = 'dejavuserif' #'stixsans'
font_scale = 1
plot_context = 'talk'
save_to = './'
ext = 'pdf'
color_palette = 'tab10'
rc_params = {
    #'mathtext.fontset': math_font, #'regular',
    #'mathtext.default': 'regular',
    'axes.formatter.use_mathtext': True,
    "font.family": font_family,  # use serif/main font for text elements
}
facet_kws = {
    'sharey': True,
    'sharex': True,
    'legend_out': True
}
legend_kws = {
    'ncol': 1,
    'frameon': True,
    'facecolor': 'white',
    'framealpha': 1.0,
    'edgecolor': 'black',
    'columnspacing': 1,
    'borderpad': 0.2,
    'handletextpad': 0.1,
    'markerscale': 3
}
fig_title_kws = {'y': 0.9, 'ha': 'center'}
sns.set_theme(
    context=plot_context,
    style='ticks',
    palette=color_palette,
    font=font_family,
    font_scale=font_scale,
    rc=rc_params
)
col_order = sorted(binding_melt['nhns'].unique())
f_grid = sns.relplot(
    data=binding_melt,
    x='rcut',
    y='averaged_bound_hns',
    hue='binding_types',
    col='nhns',
    col_order=col_order,
    col_wrap=3,
    height=height,
    aspect=aspect,
    palette=color_palette,
    legend='full',
    kind='line',
    marker='o',
    alpha=0.8,
    facet_kws=facet_kws
)

f_grid.set_titles(r'$n_{{hns}}={col_name}$')
f_grid.set_titles('')
for idx, (ax, col_name) in enumerate(zip(f_grid.axes.flat,col_order)):
    ax.text(0.1,0.8, fr"$n_{{hns}}={col_name}$", transform=ax.transAxes)    
f_grid.set_axis_labels(r'$r_{thershold}$', r'$\langle n_{m}^{bound}\rangle /n_{m}$')
legend_labels = ['Total bindings', 'Cis-bindings', 'Trans-bindings']
for idx, new_label in enumerate(legend_labels):
#    f_grid._legend.legendHandles[idx].set_linewidth(7)
    f_grid._legend.texts[idx].set_ha('left')
    f_grid._legend.texts[idx].set_text(new_label)
f_grid.set(xticks=rcut_m_hpatch_range)
f_grid.fig.suptitle(r'$N=200, D/a_m=8, a_c/a_m=2, \phi_c=0.097$', **fig_title_kws)
f_grid.tight_layout(w_pad=0.2, h_pad=0.2)
sns.move_legend(f_grid, title='', loc='upper left', bbox_to_anchor=(0.2,0.8))
#f_grid.set(yscale='log')
f_grid.savefig('choice_of_thershold-cis_trans.pdf', bbox_inches='tight', dpi=300)

In [ ]:
#Figure: cis and trans monomers
value_vars = ['n_mon_bound', 'n_mon_cis', 'n_mon_trans']
binding_melt = pd.melt(binding_df, id_vars=id_vars, value_vars=value_vars, var_name='binding_types', value_name='averaged_bound_hns')
height = 7
aspect = 1.618 # Golden ratio
font_family = 'Times New Roman'#'DejaVu Sans'#'sans-serif' # 'Times New Roman'
#math_font = 'dejavusans' #'dejavuserif' #'stixsans'
font_scale = 1.4
plot_context = 'talk'
save_to = './'
ext = 'pdf'
color_palette = 'tab10'
rc_params = {
    #'mathtext.fontset': math_font, #'regular',
    #'mathtext.default': 'regular',
    'axes.formatter.use_mathtext': True,
    "font.family": font_family,  # use serif/main font for text elements
}
legend_kws = {
    'ncol': 1,
    'frameon': True,
    'facecolor': 'white',
    'framealpha': 1.0,
    'edgecolor': 'black',
    'columnspacing': 1,
    'borderpad': 0.2,
    'handletextpad': 0.1,
    'markerscale': 3
}
fig, ax = plt.subplots(1,1, figsize=(aspect*height,height)) 
sns.set_theme(
    context=plot_context,
    style='ticks',
    palette=color_palette,
    font=font_family,
    font_scale=font_scale,
    rc=rc_params
)
hue_order = list(binding_melt['binding_types'].unique())
#hue_order = ['n_hcore_free', 'n_hcore_dangle', 'n_hcore_bridge']
col_order = sorted(binding_melt['nhns'].unique())
f_grid = sns.lineplot(
    data=binding_melt,
    x='rcut',
    y='averaged_bound_hns',
    hue='binding_types',
    hue_order=hue_order,
    style='nhns',
    style_order=col_order,
    legend='full',
    dashes=True,
    markers=True,
    estimator=None,
    markersize=15,
    alpha=0.8,
    ax=ax
)
ax.axvline(x=rcut_m_hpatch_range[0], color='black',label=r'$r_{thers}=(a_m+a_{h,patch})/2$', ls='--', zorder=0)
ax.axvline(x=rcut_m_hpatch_range[-1], color='black',label=r'$r_{thers}=2^{1/6}(a_m+a_{h,patch})/2$',ls='-.', zorder=0)
f_grid.set_title(r'$N=200, D/a_m=8, a_c/a_m=2, \phi_c=0.097$', y=1.01)
f_grid.set_xlabel(r'$r_{thers}$')
f_grid.set_ylabel(r'$\langle n_{m}^{bound}\rangle /n_{m}$')
f_grid.set(xticks=rcut_m_hpatch_range)
handles, labels = ax.get_legend_handles_labels()
legend_labels = ['Binding types'] + ['Total bindings', 'Cis-bindings', 'Trans-bindings'] + [r'$n_{hns}$'] + col_order + labels[-2:]
ax.legend(handles[1:], legend_labels[1:], loc='upper left', bbox_to_anchor=(1,1), frameon=False)
#f_grid.set(yscale='log')
fig.savefig('choice_of_thershold-cis_trans-all_in_one.pdf', dpi=300, bbox_inches='tight')